<h4>Importing libraries</h4>

In [148]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim# module to convert an address into latitude and longitude values
import requests

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML
import requests
import seaborn as sns
import matplotlib.pyplot as plt


In [147]:
!conda install -c conda-forge folium=0.5.0
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                       

<h4>Initialze API keys</h4>

In [6]:
# Foursquare api
CLIENT_ID = 'XGF5F1WVM0W1G3WDCA4MY2PFPMGPC5SRVZNDKKNHLPWSYBKJ'
CLIENT_SECRET = 'TSL2XDNGRPT5PECDN0JPIBIZVZ4UB1QLHJNVYCC52AJZRM2L'
VERSION = '20200802'
LIMIT = 3000

<h4>Fetching Cafes' details around Gurgaon city</h4> 

In [62]:
# lat lng of gurgaon
 = 28.457523
lng = 77.026344
radius = 20000 # 20km radius
query = 'cafe'

#Foursquare api call
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,
radius,LIMIT,query)
results_cafes = requests.get(url).json()

In [64]:
results_cafes

{'meta': {'code': 200, 'requestId': '5f266c8c880aab311d5e9751'},
 'response': {'venues': [{'id': '4e0580ae52b165b7d7a227cc',
    'name': 'Café Coffee Day',
    'location': {'address': '99, Huda Commercial Market',
     'crossStreet': 'Sector 32',
     'lat': 28.445862484770874,
     'lng': 77.04457084480875,
     'labeledLatLngs': [{'label': 'display',
       'lat': 28.445862484770874,
       'lng': 77.04457084480875}],
     'distance': 2206,
     'postalCode': '122001',
     'cc': 'IN',
     'city': 'Gurgaon',
     'state': 'Haryāna',
     'country': 'India',
     'formattedAddress': ['99, Huda Commercial Market (Sector 32)',
      'Gurgaon 122001',
      'Haryāna',
      'India']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': '

In [108]:

#a = results_cafes['response']['venues']
#len(a)
cafe_list_name = []
cafe_list_lat = []
cafe_list_lng = []
cafe_list_cat = []

results_cafes['response']['venues'][0]['name']
results_cafes['response']['venues'][0]['location']['lat']
results_cafes['response']['venues'][0]['location']['lng']
results_cafes['response']['venues'][0]['location']['address']
results_cafes['response']['venues'][0]['categories'][0]['name']

for v_item in results_cafes['response']['venues'] :
    cafe_list_name.append(v_item['name'])
    cafe_list_lat.append(v_item['location']['lat'])
    cafe_list_lng.append(v_item['location']['lng'])    
    for cat_item in v_item['categories']:
        cafe_list_cat.append(cat_item['name'])
        
cafe_list

df_cafes = pd.DataFrame(list(zip(cafe_list_name,cafe_list_lat,cafe_list_lng,cafe_list_cat)),columns = ['name','latitude','longitude','category'])
df_cafes.head(5)


,name,latitude,longitude,category
0,Café Coffee Day,28.445862,77.044571,Coffee Shop
1,Cafe Coffee Day (CCD),28.453253,77.050540,Café
2,Cafe Coffee Day,28.443509,77.055898,Café
3,Café G,28.468331,77.059575,Indian Restaurant
4,Cafe Soho,28.450522,77.031884,Diner


<h4> Analysing the dataset </h4>

In [116]:
catg = df_cafes.groupby('category')
catg.count()

,name,latitude,longitude
category,,,
Asian Restaurant,1,1,1
Automotive Shop,1,1,1
Bakery,2,2,2
Bistro,1,1,1
Café,21,21,21
Coffee Shop,9,9,9
Dance Studio,1,1,1
Diner,1,1,1
Fast Food Restaurant,1,1,1


In [142]:
# select only cafes & coffee shop
df_cafes_fltr = df_cafes[(df_cafes['category'] == 'Café') | (df_cafes['category'] == 'Coffee Shop')]
df_cafes_fltr.head()

,name,latitude,longitude,category
0,Café Coffee Day,28.445862,77.044571,Coffee Shop
1,Cafe Coffee Day (CCD),28.453253,77.050540,Café
2,Cafe Coffee Day,28.443509,77.055898,Café
5,Cafe Coffee Day (CCD),28.468912,77.062180,Café
8,Cafe Coffee Day (CCD),28.416664,77.041017,Coffee Shop


In [143]:
df_cafes_fltr.shape

(30, 4)

<h4> Visualizing the data </h4>

In [160]:
cafe_ggn_map = folium.Map(location = [lat,lng],zoom_start= 13)

for lat, lng, name in zip(df_cafes['latitude'],df_cafes['longitude'],df_cafes['name']):
    
    label = folium.Popup(name, parse_html = True)
    
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(cafe_ggn_map)  
    
cafe_ggn_map

<h4>Cluster locations</h4>

In [163]:
df_cafes_K = df_cafes_fltr.drop(['name','category'], axis = 1)
df_cafes_K

,latitude,longitude
0,28.445862,77.044571
1,28.453253,77.050540
2,28.443509,77.055898
5,28.468912,77.062180
8,28.416664,77.041017
9,28.472436,77.072627
11,28.467584,77.081695
12,28.508034,77.030053
13,28.467648,77.081393
16,28.481362,77.093001


In [207]:
Kclusters = 4

kmeans = KMeans(n_clusters=Kclusters,random_state=0).fit(df_cafes_K)


In [208]:
kmeans.labels_
df_cafes_fltr.drop('Cluster Labels',axis = True, inplace = True)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [209]:
#cafe_label = pd.DataFrame(kmeans.labels_ , columns = ['cluster label'])
df_cafes_fltr.insert(0, 'Cluster Labels', kmeans.labels_)

In [210]:

df_cafes_fltr.head()

,Cluster Labels,name,latitude,longitude,category
0,2,Café Coffee Day,28.445862,77.044571,Coffee Shop
1,2,Cafe Coffee Day (CCD),28.453253,77.050540,Café
2,2,Cafe Coffee Day,28.443509,77.055898,Café
5,2,Cafe Coffee Day (CCD),28.468912,77.062180,Café
8,1,Cafe Coffee Day (CCD),28.416664,77.041017,Coffee Shop


<h4>Visualize the clusters</h4>

In [211]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[lat, lng], zoom_start=13)

# set color scheme for the clusters
x = np.arange(Kclusters)
ys = [i + x + (i*x)**2 for i in range(Kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(df_cafes_fltr['latitude'], df_cafes_fltr['longitude'], df_cafes_fltr['Cluster Labels']):
    label = folium.Popup( ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters